In [1]:
# http://localhost:8080/notebooks/git/product-category/notebooks/prep_20210302A1.ipynb
prfx_prp = 'prep_20210302B1'

In [2]:
import pandas as pd
import numpy as np
from collections import Counter


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier




In [3]:
MIN_CNT = 50

In [21]:
%%time
df = pd.read_csv(f'../data/data__{prfx_prp}.csv', nrows=1000000)

CPU times: user 6.79 s, sys: 463 ms, total: 7.26 s
Wall time: 7.25 s


In [22]:
foo = df.category.apply(lambda x: eval(x))

SyntaxError: invalid syntax (<string>, line 1)

In [24]:
for i,v in enumerate(df.category):
    i,eval(v)

SyntaxError: invalid syntax (<string>, line 1)

In [26]:
df.iloc[i]

category       2008 2009 2010 2011 2012 2013 Jeep Grand Chero...
description                                                  NaN
title                                                        NaN
brand                                                        NaN
feature                                                      NaN
asin                                                         NaN
domain                                                       NaN
Name: 562054, dtype: object

In [28]:
def try2eval(x):
    try:
        return eval(x)
    except SyntaxError:
        return []

In [29]:
foo = df.category.apply(try2eval)

In [27]:
df.head()

,category,description,title,brand,feature,asin,domain
0,"['Appliances', 'Refrigerators, Freezers & Ice ...",NaN,Tupperware Freezer Square Round Container Set ...,Tupperware,Each 3-pc. set includes two 7/8-cup/200 mL and...,7301113188,Appliance
1,"['Appliances', 'Refrigerators, Freezers & Ice ...",2 X Tupperware Pure & Fresh Unique Covered Coo...,2 X Tupperware Pure &amp; Fresh Unique Covered...,Tupperware,2 X Tupperware Pure & Fresh Unique Covered Coo...,7861850250,Appliance
2,"['Appliances', 'Parts &amp; Accessories']",NaN,The Cigar - Moments of Pleasure,The Cigar Book,NaN,8792559360,Appliance
3,"['Appliances', 'Parts & Accessories']","Multi purpost descaler, especially suited to w...",Caraselle 2X 50G Appliance Descalene,Caraselle,NaN,9792954481,Appliance
4,"['Appliances', 'Parts & Accessories', 'Range P...",Full gauge and size beveled-edge; furnished wi...,EATON Wiring 39CH-SP-L Arrow Hart 1-Gang Chrom...,EATON Wiring,Returns will not be honored on this closeout i...,B00002N5EL,Appliance


In [15]:
# df = pd.read_csv(f'../data/data_sample__{prfx_prp}.csv')
df['category'] = df.category.apply(eval)
df.fillna("", inplace=True)
df = df.astype(str)
print("df.shape:", df.shape)
df['txt'] = df.title + " " + df.brand + " " + df.description + " " + df.feature

SyntaxError: invalid syntax (<string>, line 1)

In [5]:
dmn2cnt = Counter(df.domain.value_counts().to_dict())
i2dmn = sorted(dmn2cnt.keys())
dmn2i = {v:k for k,v in enumerate(i2dmn)}
cat2cnt = Counter([j for i in df.category for j in i])
i2cat = sorted(k for k,v in cat2cnt.items() if v>50)
cat2i = {v:k for k,v in enumerate(i2cat)}

len(i2dmn), len(i2cat)

(25, 123)

## split train val

In [6]:
np.random.seed(101)
msk_val = np.random.rand(len(df))>0.85
dftrn = df[~msk_val]
dfval = df[msk_val]
dftrn.shape, dfval.shape

((13264290, 7), (2341415, 7))

## rf model

In [7]:
%%time
vec = TfidfVectorizer(ngram_range=(1,2),
                      min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
                      smooth_idf=1, sublinear_tf=1)
Xtrn = vec.fit_transform(dftrn.txt)
Xval = vec.transform(dfval.txt)

Xtrn.shape, Xval.shape

AttributeError: 'DataFrame' object has no attribute 'txt'

In [8]:
df['cat'] = df.category.apply(lambda x: [cat2i[o]])

NameError: name 'o' is not defined

In [ ]:
ys = np.zeros((len(df), len(i2cat)))

for i,cats in enumerate(df.category):
    idx_pos = [cat2i[cat] for cat in cats if cat in cat2i]
    ys[i,idx_pos] = 1

ys_trn = ys[~msk_val]
ys_val = ys[msk_val]

In [ ]:
clf = RandomForestClassifier(n_estimators=5, 
                             min_samples_leaf=2,
                             max_features=0.5,
                             n_jobs=-1)

In [ ]:
%%time
clf.fit(Xtrn, ys_trn)

In [ ]:
clf.n_outputs_

In [ ]:
prds_trn = clf.predict(Xtrn)
prds_prob_trn = clf.predict_proba(Xtrn)

np.mean(prds_trn==ys_trn), accuracy_score(ys_trn, prds_trn)

In [ ]:
prds_val = clf.predict(Xval)
prds_prob_val = clf.predict_proba(Xval)

np.mean(prds_val==ys_val), accuracy_score(ys_val, prds_val)

In [ ]:
for j in range(ys_val.shape[1]):
    print(accuracy_score(ys_val[:,j], prds_val[:,j]))